In [1]:
from lxml import html  
import csv,os,json
import requests

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
import pandas as pd
import numpy as np
import time
from time import sleep

In [2]:
keyword_df=pd.read_csv('Keywords.csv')

#Delete the duplicate of the keyword
keyword_df=keyword_df.drop_duplicates(keep='first')
print(keyword_df.describe())

#Separate the keyword in terms of words count
#keyword_1 represent the keyword that contains only one word
#keyword_2 represent the keyword that contains two words, etc
keyword_1=[]
keyword_2=[]
keyword_3=[]
keyword_4=[]
for i in range(0,2783):
    if len(keyword_df.iloc[i,0].split(' '))==1:
        keyword_1.append(keyword_df.iloc[i,0])
    elif len(keyword_df.iloc[i,0].split(' '))==2:
        keyword_2.append(keyword_df.iloc[i,0])
    elif len(keyword_df.iloc[i,0].split(' '))==3:
        keyword_3.append(keyword_df.iloc[i,0])
    elif len(keyword_df.iloc[i,0].split(' '))==4:
        keyword_4.append(keyword_df.iloc[i,0])
print("Number of the keyword that has one word: "+str(len(keyword_1)))
print("Number of the keyword that has two words: "+str(len(keyword_2)))
print("Number of the keyword that has three words: "+str(len(keyword_3)))
print("Number of the keyword that has four words: "+str(len(keyword_4)))

                    col
count              2783
unique             2783
top     nike golf shoes
freq                  1
Number of the keyword that has one word: 17
Number of the keyword that has two words: 1071
Number of the keyword that has three words: 936
Number of the keyword that has four words: 510


In [5]:
"""
Using item url to visit webpage of each product and parse its name, price, availability, star and review number
"""
def get_info(item_url):
    
    #To avoid being blocked by Amazon's website, we create a header txt that contains a list of user agents.
    #And then randomly change the user agents
    header_file = 'header.txt'
    with open(header_file) as f:
        lines = f.readlines()
    random_header=np.random.choice(lines).split('\n')[0]
    headers={'User-Agent': random_header}
    page = requests.get(item_url,headers=headers)
    
    #Give webpage time to load the information when visit the page.
    #It also helps to avoid being blocked by website.
    delays = [7, 3, 2, 10, 16]
    delay = np.random.choice(delays)
    sleep(delay)
    tree = html.fromstring(page.content)
    
    #Using xpath address to locate the certain information in terms of html structure
    xpath_name="//h1[@id='title']//text()" 
    xpath_price="//span[contains(@id,'priceblock_dealprice') or contains(@id,'priceblock_ourprice')]/text()"
    xpath_availability="//div[@id='availability']//text()"
    xpath_rev="//a[@id='acrCustomerReviewLink']//text()"
    xpath_star="//span[@id='acrPopover']//text()"
    xpath_tag="//span[@id='AMAZONS_CHOICE_B003EM8008-label']//text()"
    xpath_type_of_sale="//div[@id='merchant-info']//text()"
    xpath_category="//a[@class='a-link-normal a-color-tertiary']//text()"
    
    raw_name=tree.xpath(xpath_name)
    raw_price=tree.xpath(xpath_price)
    raw_availability=tree.xpath(xpath_availability)
    raw_rev=tree.xpath(xpath_rev)[0:3]
    raw_star=tree.xpath(xpath_star)
    raw_type_of_sale=tree.xpath(xpath_type_of_sale)
    raw_category=tree.xpath(xpath_category)
    raw_tag=tree.xpath(xpath_tag)
    
    #Transform the scraped information into string format
    #If the information cannot be scrapped, we assign None value to the variable
    name=' '.join(''.join(raw_name).split()) if raw_name else None
    price=' '.join(''.join(raw_price).split()).strip() if raw_price else None
    availability=' '.join(''.join(raw_availability).split()).strip() if raw_availability else None
    review=' '.join(''.join(raw_rev).split()) if raw_rev else None
    star=' '.join(''.join(raw_star).split()) if raw_star else None
    type_of_sale=' '.join(''.join(raw_type_of_sale).split()) if raw_type_of_sale else None
    category=' '.join(''.join(raw_category).split()) if raw_category else None
    
    #Save all scraped information into a format of dictionary
    data={
        'Name':name,
        'Price':price,
        'Availability':availability,
        'Asin_number':item_url.replace("http://www.amazon.com/dp/", ""),
        'Number of Reviews':review,
        'Star':star,
        'Type of sale':type_of_sale,
        'Category':category,
    } 
 
    return data

"""
Using asin number to parse product's name, price, availability, review from item page
"""
def read_asin(search_url,asin_list):

    extracted_data_per_page = []
    for item in asin_list:
        item_url = "http://www.amazon.com/dp/"+item
        print ("Processing: "+item_url)
        extracted_data_per_page.append(get_info(item_url))
        
    return extracted_data_per_page

def save_data(extracted_data_all):
    
    f=open('extracted_data_demo.json','w')
    json.dump(extracted_data_all,f,indent=4)

"""
Get asin number and tag of each item from the searching page
"""
def get_asin_tag(url):
    
    asin_list=[]
    tag_list=[]
    
    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'}
    page = requests.get(url,headers=headers)
    tree = html.fromstring(page.content)
    
    lis_num=0
    #Scraped each page that has 20 records/items
    for i in range(lis_num,lis_num+20): 
    
        lists=driver.find_elements_by_xpath("//li[@id='result_%d']"%i)
    
        for idx,item in enumerate(lists):    
            div_asin=item.get_attribute("data-asin")       
            
            asin_list.append(div_asin)
        try:
            tag_css="li#result_%d"%i +" "+"a.a-badge span" 
            tag=driver.find_element_by_css_selector(tag_css)
            tag_list.append(tag.text) 
        except selenium.common.exceptions.NoSuchElementException:
            tag_list.append('None')
    
    return asin_list, tag_list
    
if __name__ == "__main__":
    
    extracted_data_all=[]
    #Control the scraped number of keywords in list
    #For demo, here we set 0 to 2 to scraped 2 keywords in total. It may take 7 minutes to finish scraping.
    #In practice, we scraped 100 keywords for around 5 hrs per time
    for item in keyword_2[0:2]:
        keyword=item.split(" ")           
        search_url="https://www.amazon.com/s/ref=nb_sb_noss_1?url=search-alias%3Daps&field-keywords="+keyword[0]+"+"+keyword[1]
        executable_path = 'driver_mac/geckodriver'
        driver = webdriver.Firefox(executable_path=executable_path)
        main_search=driver.get(search_url)
        
        print("Current searching item: "+ keyword[0]+" "+keyword[1])
        asin_list, tag_list=get_asin_tag(search_url)
        
        #We only scraped the item that has 20 records on first searching page
        if len(asin_list)==20:
            
            extracted_data_per_page=read_asin(search_url,asin_list)
            for i in range(0,20):
                extracted_data_per_page[i].update({'Tag':tag_list[i]})
            extracted_data_all=extracted_data_all+extracted_data_per_page
        
        driver.quit()
        
    save_data(extracted_data_all)

Current searching item: water table
Processing: http://www.amazon.com/dp/B01K1K0K6M
Processing: http://www.amazon.com/dp/B077LPBCGP
Processing: http://www.amazon.com/dp/B0193MPOAQ
Processing: http://www.amazon.com/dp/B01K1K0K6M
Processing: http://www.amazon.com/dp/B004INDQWY
Processing: http://www.amazon.com/dp/B07BFQDHH4
Processing: http://www.amazon.com/dp/B00QGSMK8K
Processing: http://www.amazon.com/dp/B00ID0Y4GK
Processing: http://www.amazon.com/dp/B00B0DWB62
Processing: http://www.amazon.com/dp/B01LXS9TD1
Processing: http://www.amazon.com/dp/B00IGXUSGY
Processing: http://www.amazon.com/dp/B077ZGZZMB
Processing: http://www.amazon.com/dp/B0193MPQK4
Processing: http://www.amazon.com/dp/B01K1L1IX0
Processing: http://www.amazon.com/dp/B0742GRWVX
Processing: http://www.amazon.com/dp/B077LG9CP6
Processing: http://www.amazon.com/dp/B01FQTFR34
Processing: http://www.amazon.com/dp/B01M5DAMBV
Processing: http://www.amazon.com/dp/B00REOJ9AC
Processing: http://www.amazon.com/dp/B00IGXUSBE
Curr

In [9]:
data_df = pd.read_json('extracted_data_demo.json')
data_df.tail(20)

,Asin_number,Availability,Category,Name,Number of Reviews,Price,Star,Tag,Type of sale
20,B06WD82BK8,In Stock.,Sports & Outdoors Outdoor Recreation Camping &...,Portable Height Adjustable Aluminum Folding Ca...,22 customer reviews,$39.99,3.8 out of 5 stars,None,Sold by Hwabond Luxuries and Fulfilled by Amaz...
21,B017W26Q9C,In Stock.,Home & Kitchen Furniture Game & Recreation Roo...,"COSCO Products 20"" x 48"" Resin Top Folding Tab...",611 customer reviews,$24.39,3.9 out of 5 stars 3.9 out of 5 stars,None,Ships from and sold by Amazon.com.
22,B003YJPC2A,In Stock.,"Patio, Lawn & Garden Patio Furniture & Accesso...",Lifetime 4428 Height Adjustable Folding Utilit...,"3,292 customer reviews",$37.07,4.5 out of 5 stars 4.5 out of 5 stars,None,Ships from and sold by Amazon.com. Gift-wrap a...
23,B003DYZQKY,In Stock.,Home & Kitchen Furniture Game & Recreation Roo...,Lifetime 80160 Commercial Height Adjustable Fo...,"2,016 customer reviews",$39.41,4.4 out of 5 stars,Amazon's Choice,Ships from and sold by Amazon.com. Gift-wrap a...
24,B007RP7JU0,None,None,None,None,None,None,None,None
25,B0741NQ23J,Usually ships within 2 to 3 days.,Home & Kitchen Furniture Game & Recreation Roo...,"COSCO Diamond Series 300 lb. Weight Capacity, ...",100 customer reviews,$64.99,4.5 out of 5 stars 4.5 out of 5 stars,None,Ships from and sold by Amazon.com. Gift-wrap a...
26,B017W26Q9C,In Stock.,Home & Kitchen Furniture Game & Recreation Roo...,"COSCO Products 20"" x 48"" Resin Top Folding Tab...",611 customer reviews,$24.39,3.9 out of 5 stars 3.9 out of 5 stars,None,Ships from and sold by Amazon.com.
27,B00DOZTM3O,In Stock.,Home & Kitchen Furniture Game & Recreation Roo...,Cosco 14678BLK1 Deluxe 6 Foot x 30 inch Half B...,"1,358 customer reviews",$50.00,4.3 out of 5 stars 4.3 out of 5 stars,None,Ships from and sold by PLENTY OF THAT.
28,B07BQFYHKG,None,None,None,None,None,None,None,None
29,B00D49B0US,In Stock.,"Patio, Lawn & Garden Patio Furniture & Accesso...",Best ChoiceProducts Folding Table Portable Pla...,719 customer reviews,$67.99,4.0 out of 5 stars,Best Seller,Ships from and sold by BestChoiceproducts.
